In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning) 
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

njobs = -1

In [11]:
train=pd.read_csv('train.csv',sep='|')
test=pd.read_csv('test.csv',sep='|')

In [12]:
print(train.fraud.value_counts())
print(train.fraud.value_counts() / len(train))

0    1775
1     104
Name: fraud, dtype: int64
0    0.944651
1    0.055349
Name: fraud, dtype: float64


# Feature Engineering

In [13]:
# for training set

#train['totalScanTimeInMinutes'] = train['totalScanTimeInSeconds'] / 60.0
train['scannedLineItems'] = train['scannedLineItemsPerSecond'] * train['totalScanTimeInSeconds']
train['pricePerScannedLineItem'] = train['grandTotal'] / train['scannedLineItems']
train['scansWithoutRegistrationPerScannedLineItem'] = train['scansWithoutRegistration'] / train['scannedLineItems']
train['quantityModificationsPerScannedLineItem'] = train['quantityModifications'] / train['scannedLineItems']
train['lineItemVoidsPerSecond'] = train['lineItemVoids'] / train['totalScanTimeInSeconds']
train['scansWithoutRegistrationPerSecond'] = train['scansWithoutRegistration'] / train['totalScanTimeInSeconds']
train['quantityModificationsPerSecond'] = train['quantityModifications'] / train['totalScanTimeInSeconds']
train['secondsPerEuro'] = train['totalScanTimeInSeconds'] / train['grandTotal']
train['lineItemVoidsPerEuro'] = train['lineItemVoids'] / train['grandTotal']
train['scansWithoutRegistrationPerEuro'] = train['scansWithoutRegistration'] / train['grandTotal']
train['quantityModificationsPerEuro'] = train['quantityModifications'] / train['grandTotal']


# for test set

#train['totalScanTimeInMinutes'] = train['totalScanTimeInSeconds'] / 60.0
test['scannedLineItems'] = test['scannedLineItemsPerSecond'] * test['totalScanTimeInSeconds']
test['pricePerScannedLineItem'] = test['grandTotal'] / test['scannedLineItems']
test['scansWithoutRegistrationPerScannedLineItem'] = test['scansWithoutRegistration'] / test['scannedLineItems']
test['quantityModificationsPerScannedLineItem'] = test['quantityModifications'] / test['scannedLineItems']
test['lineItemVoidsPerSecond'] = test['lineItemVoids'] / test['totalScanTimeInSeconds']
test['scansWithoutRegistrationPerSecond'] = train['scansWithoutRegistration'] / train['totalScanTimeInSeconds']
test['quantityModificationsPerSecond'] = train['quantityModifications'] / train['totalScanTimeInSeconds']
test['secondsPerEuro'] = train['totalScanTimeInSeconds'] / train['grandTotal']
test['lineItemVoidsPerEuro'] = train['lineItemVoids'] / train['grandTotal']
test['scansWithoutRegistrationPerEuro'] = train['scansWithoutRegistration'] / train['grandTotal']
test['quantityModificationsPerEuro'] = train['quantityModifications'] / train['grandTotal']

In [ ]:
from sklearn.tree import ExtraTreeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm.classes import OneClassSVM
from sklearn.neural_network.multilayer_perceptron import MLPClassifier
from sklearn.neighbors.classification import RadiusNeighborsClassifier
from sklearn.neighbors.classification import KNeighborsClassifier
from sklearn.multioutput import ClassifierChain
from sklearn.multioutput import MultiOutputClassifier
from sklearn.multiclass import OutputCodeClassifier
from sklearn.multiclass import OneVsOneClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model.stochastic_gradient import SGDClassifier
from sklearn.linear_model.ridge import RidgeClassifierCV
from sklearn.linear_model.ridge import RidgeClassifier
from sklearn.linear_model.passive_aggressive import PassiveAggressiveClassifier
from sklearn.gaussian_process.gpc import GaussianProcessClassifier
from sklearn.ensemble.voting_classifier import VotingClassifier
from sklearn.ensemble.weight_boosting import AdaBoostClassifier
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier
from sklearn.ensemble.bagging import BaggingClassifier
from sklearn.ensemble.forest import ExtraTreesClassifier
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import GaussianNB
from sklearn.semi_supervised import LabelPropagation
from sklearn.semi_supervised import LabelSpreading
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import NearestCentroid
from sklearn.svm import NuSVC
from sklearn.linear_model import Perceptron
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
# from sklearn.mixture import DPGMM
#from sklearn.mixture import GMM
#from sklearn.mixture import GaussianMixture
#from sklearn.mixture import VBGMM



model_factory = [
# RandomForestRegressor(),
# XGBRegressor(nthread=1),
 #MLPRegressor(),
# Ridge(),
# BayesianRidge(),
# ExtraTreesRegressor(),
# ElasticNet(),
# KNeighborsRegressor(),
# GradientBoostingRegressor()
  ExtraTreeClassifier(),
    DecisionTreeClassifier(),
 #   OneClassSVM(),  # doesn't work instantly
    
                  # MLPClassifier(),# takes very long for a larger training set
    
 #   RadiusNeighborsClassifier(), # doesn't work instantly
    KNeighborsClassifier(),
#    ClassifierChain(),     # ensemble method
#    MultiOutputClassifier(), # ensemble method
#    OutputCodeClassifier(), # ensemble method
#    OneVsOneClassifier(), # ensemble method
#    OneVsRestClassifier(), # ensemble method
    SGDClassifier(),
    RidgeClassifierCV(),
    RidgeClassifier(),
    PassiveAggressiveClassifier(),
    
                   # GaussianProcessClassifier(),  # takes very long for a larger training set
    
#    VotingClassifier(), # ensemble method
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    BaggingClassifier(),
    ExtraTreesClassifier(),
    RandomForestClassifier(),
    BernoulliNB(),
  #  CalibratedClassifierCV(), # doesn't work instantly
    GaussianNB(), # causing some problems
 #   LabelPropagation(), # doesn't work instantly
 #   LabelSpreading(),  # doesn't work instantly
    LinearDiscriminantAnalysis(), # causing some problems
 #   LinearSVC(max_iter = 100000), # causing some problems
    LogisticRegression(max_iter = 10000),
    LogisticRegressionCV(max_iter = 10000),
    MultinomialNB(),
    NearestCentroid(),
  #  NuSVC(),   # doesn't work instantly
    Perceptron(),
    QuadraticDiscriminantAnalysis(),
    # SVC(),
#    DPGMM(),
#    GMM(),
#    GaussianMixture(),
#    VBGMM()
]

from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer


def my_custom_loss_func(y_true, y_pred):
    cm = confusion_matrix(y_true,y_pred)
    tn, fp, fn, tp = cm.ravel()
    score = ((-25)*fp + (-5)*fn + 5*tp) / len(y_true)
    return (score)

my_custom_score = make_scorer(my_custom_loss_func, greater_is_better=True)

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut

skf = StratifiedKFold(n_splits=10)



from sklearn.model_selection import GridSearchCV


model_tuning_factory = [
    GridSearchCV(LogisticRegression(max_iter = 10000), 
                 dict(# penalty = ['l1','l2'],  # automatic regularization  -> option 'l1' doesnt work with all solvers and leads to errors
                      fit_intercept = [True, False]),
                    #  solver = ['lbfgs', 'liblinear', 'sag', 'saga']),
                 cv = skf,
                 scoring = my_custom_score),
    GridSearchCV(LogisticRegressionCV(max_iter = 10000), 
                 dict(# penalty = ['l1','l2'],  # automatic regularization  -> option 'l1' doesnt work with all solvers and leads to errors
                      fit_intercept = [True, False]),
                    #  solver = ['lbfgs', 'liblinear', 'sag', 'saga']),
                 cv = skf,
                 scoring = my_custom_score),
    GridSearchCV(KNeighborsClassifier(), 
                 dict(n_neighbors = range(1,4),
                      weights = ['uniform', 'distance']),
                 cv = skf,
                 scoring = my_custom_score),
    GridSearchCV(DecisionTreeClassifier(),
                 dict(criterion = ['entropy', 'gini']),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score),
    GridSearchCV(ExtraTreeClassifier(),
                 dict(criterion = ['entropy', 'gini']),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score),
    GridSearchCV(AdaBoostClassifier(),
                 dict(n_estimators = range(1,100)),
                      #max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score),    
    GridSearchCV(BernoulliNB(),
                 dict(binarize  = np.arange(0.0, 1.0, 0.1)),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score), 
    GridSearchCV(GaussianNB(),
                 dict(),
                     #var_smoothing  = np.arange(0.0, 1.0, 0.1),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score), 
                 
    GridSearchCV(GradientBoostingClassifier(),
                 dict(learning_rate = np.arange(0.01, 1.0, 0.01),
                      n_estimators = range(50,150)),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score),                  
                 
    GridSearchCV(BaggingClassifier(), 
                 dict(n_estimators = range(5,50)),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score), 
    GridSearchCV(NearestCentroid(),
                 dict(),
                 # dict(n_estimators = range(5,50)),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score), 
    GridSearchCV(Perceptron(),
                 dict(),
                 # dict(n_estimators = range(5,50)),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score), 
    GridSearchCV(LinearDiscriminantAnalysis(),
                 dict(),
                 #dict(n_components = range(5,50)),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score),
    GridSearchCV(QuadraticDiscriminantAnalysis(),
                 dict(),
                 # dict(n_estimators = range(5,50)),
                     # max_depth = range(1,100)),
                 #     max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score)                 
]


from sklearn.model_selection import RandomizedSearchCV

model_tuning_factory_temp = [
    GridSearchCV(DecisionTreeClassifier(),
                 dict(criterion = ['entropy', 'gini'],
                     # max_depth = range(1,100)),
                     #  max_leaf_nodes = range(2,100)),
                 cv = skf,
                 scoring = my_custom_score),
    GridSearchCV(LogisticRegression(max_iter = 10000), 
                 dict(# penalty = ['l1','l2'],  # automatic regularization  -> option 'l1' doesnt work with all solvers and leads to errors
                      fit_intercept = [True, False]),
                    #  solver = ['lbfgs', 'liblinear', 'sag', 'saga']),
                 cv = skf,
                 scoring = my_custom_score)
]



model_tuning_factory2 = [
    RandomizedSearchCV(KNeighborsClassifier(), 
                 dict(n_neighbors = range(1,4),
                      weights = ['uniform', 'distance']),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = 10),
    RandomizedSearchCV(LogisticRegression(max_iter = 10000), 
                 dict(#penalty = ['l2'],  # automatic regularization  -> option 'l1' doesnt work with all solvers and leads to errors
                      fit_intercept = [True, False]),
                      #solver = ['lbfgs', 'liblinear', 'sag', 'saga']),
                 cv = skf,
                 scoring = my_custom_score,
                 n_iter = 10)
]

In [28]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer

def my_custom_loss_func(y_true, y_pred):
    cm = confusion_matrix(y_true,y_pred)
    tn, fp, fn, tp = cm.ravel()
    score = ((-25)*fp + (-5)*fn + 5*tp) / len(y_true)
    return (score)

my_custom_score = make_scorer(my_custom_loss_func, greater_is_better=True)

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import LeaveOneOut

skf = StratifiedKFold(n_splits=10)
loo = LeaveOneOut()

In [71]:
Y = train['fraud']
X = train.drop('fraud',axis=1)

In [ ]:
import time
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_classif

from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import StandardScaler

pd.DataFrame(feature_scaler.fit_transform(train.values), columns=train.columns, index=train.index)

result_table = pd.DataFrame(columns=["Model", "Data Preparation", "Feature Count", "Features", "Optimal Parameters", "Monetary Value Per Instance - Mean", "Monetary Value Per Instance - Standard Deviation", "Raw Model"])


# three types of data preparation: No preparation, MaxMinScaler, StandardScaler
for data_preparation_step in range(1,4):
    if (data_preparation_step == 1):  
        X_scaled = X
        data_preparation = "None"
    elif (data_preparation_step == 2):
        feature_scaler = MinMaxScaler()  
        feature_scaler.fit_transform(X.values) 
        X_scaled = pd.DataFrame(feature_scaler.fit_transform(X.values), columns=X.columns, index=X.index) 
        data_preparation = "MinMaxScaler"
    elif (data_preparation_step == 3):
        feature_scaler = StandardScaler()  
        feature_scaler.fit_transform(X.values) 
        X_scaled = pd.DataFrame(feature_scaler.fit_transform(X.values), columns=X.columns, index=X.index)
        data_preparation = "StandardScaler"

    for feature_count in range(1,len(list(X))+1):

        for model in model_tuning_factory_temp:   # replace with GridSearch for greater accuracy
                
            best_features = SelectKBest(f_classif, k=feature_count).fit(X_scaled,Y)
            best_feature_list = X.columns[best_features.get_support()]
            X_selected_features = X[best_feature_list]
            
            model.seed = 42
            model.fit(X_selected_features,Y)  
            model_name = model.best_estimator_.__class__.__name__
            score_mean = model.cv_results_['mean_test_score'][model.best_index_]
            score_std = model.cv_results_['std_test_score'][model.best_index_]
            
            print("Finished " + model.best_estimator_.__class__.__name__ + " with " + data_preparation + " and " + str(feature_count) + " features")

    #  print("Finished " + model.__class__.__name__ + " with " + str(feature_count) + " features:" + " after " + str(time.time() - t) + " seconds")
    #  print("Monetary Value Per Instance - Mean " + str(score_mean))
    #  print("Monetary Value Per Instance - Standard Deviation " + str(score_std))  
    #  print("")  

            result_table = result_table.append({
             "Model": model_name,
             "Data Preparation": data_preparation,
             "Feature Count": feature_count,
             "Features": best_feature_list.values,
             "Optimal Parameters": model.best_params_,
             "Monetary Value Per Instance - Mean":  score_mean,
             "Monetary Value Per Instance - Standard Deviation": score_std,
             "Raw Model": model.best_estimator_
              }, ignore_index=True)
    
result_table.sort_values(by = "Monetary Value Per Instance - Mean", ascending = False)

Finished DecisionTreeClassifier with None and 1 features
Finished LogisticRegression with None and 1 features
